# HPI

The aim of this work is to identify the clusters containing HPI genes. We blast HPI sequences on a database constituted of one sequence per cluster. We align the different hits with HPI sequence and we record sequence similarity. As we are also looking for gene fragments we make a distinction between the global similarity (proportion of matches between both sequences once aligned) and the cluster similarity (proportion of matches on the proportion of the alignment that starts with the first base of the cluster sequence and that ends with its last base i.e. we remove the gaps at the beginning and the end of the alignment of the cluster sequence).

## Libraries imports

In [2]:
import pickle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML       
from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO

## Data

In [22]:
DATA_FOLDER = "../Data/Sequences/"
REFSEQ_FOLDER = DATA_FOLDER+"Clusters reference sequences/"
HPI_FOLDER = DATA_FOLDER+"HPI sequences/"
HPI_OUTPUTS = "../Data/HPI outputs/"

CLUSTERS_GAPS_FILE = REFSEQ_FOLDER+"refseq.fasta" # Clusters reference sequences (fasta file)
CLUSTERS_FILE= REFSEQ_FOLDER+"clusters.faa" #  Clusters reference sequences without gaps (fasta file)
SINGLETONS_FILE = REFSEQ_FOLDER+"singletons.fasta" # Singletons sequences (fasta file)

HPI_FILE = HPI_FOLDER+"HPI_db.fas" # HPI sequences (fasta file)
BLAST_HPI_CLUSTERS = HPI_FOLDER+"HPI_clusters.xml" # BLAST output (HPI blasted against clusters database)
BLAST_HPI_SINGLETONS = HPI_FOLDER+"HPI_singletons.xml" # BLAST output (HPI blasted against singletons database)

ANNOTATIONS_CLUSTERS_CSV = HPI_OUTPUTS+"clusters_hits.csv"
ANNOTATIONS_SINGLETONS_CSV = HPI_OUTPUTS+"singletons_hits.csv"
ANNOTATIONS_HPI_CSV = HPI_OUTPUTS+"HPI_hits.csv"
UNIQUE_HPI_HITS_CSV = HPI_OUTPUTS+"Unique_HPI_hits.csv"
RESULTS_CLUSTERS = HPI_OUTPUTS+"results_clusters" # pickle file to save clusters hits results (to load run `results = pickle.load(open(RESULTS_CLUSTERS, 'rb'))`)
RESULTS_SINGLETONS = HPI_OUTPUTS+"results_singletons" # pickle file to save singletons hits results (to load run `results = pickle.load(open(RESULTS_SINGLETONS, 'rb'))`)

## Parameters

In [6]:
# BLAST e-value threshold
E_VALUE_THRESH = 0.00000001

# Similarity threshold between cluster sequence and gene sequence once aligned
SIMILARITY_THRESHOLD = 0.9

# Good alignment criteria: no more than max(MAX_GAPS_PROP*total number of characters,MAX_GAPS_ABS) gaps.
MAX_GAPS_ABS = 3
MAX_GAPS_PROP = 1/300 

In [7]:
SINGLETONS_FASTA = SeqIO.to_dict(SeqIO.parse(SINGLETONS_FILE, "fasta"))    
CLUSTERS_FASTA = SeqIO.to_dict(SeqIO.parse(CLUSTERS_FILE, "fasta"))   
SEQUENCES_FASTA = dict(SINGLETONS_FASTA, **CLUSTERS_FASTA)
HPI_FASTA = SeqIO.to_dict(SeqIO.parse(HPI_FILE, "fasta"))

## Blast hit class

We define a python class to record hits characteristics easily.

In [8]:
class Hit:
    similarity_threshold = SIMILARITY_THRESHOLD
    gene_dict = HPI_FASTA
    cluster_dict = SEQUENCES_FASTA
    
    def __init__(self, gene_name, cluster_name, global_similarity, cluster_similarity, start_gap, end_gap):
        self.gene_name = gene_name
        self.cluster_name = cluster_name
        self.global_similarity = global_similarity
        self.cluster_similarity = cluster_similarity
        self.start_gap = start_gap
        self.end_gap = end_gap
        
    def get_gene_name(self):
        description = self.gene_dict[self.gene_name].description
        if("[gene=" in description):
            return description.split("[gene=")[1].split("]")[0]
        else:
            return ""

    def get_gene_length(self):
        return len(Seq((str(self.gene_dict[self.gene_name].seq))))
    
    def get_cluster_length(self):
        return len(Seq((str(self.cluster_dict[self.cluster_name].seq))))

    def is_similar(self):
        return self.global_similarity > self.similarity_threshold
    
    def is_fragment(self):
        return self.get_cluster_length()<=0.9*self.get_gene_length() and self.cluster_similarity > self.similarity_threshold


## Parsing BLAST results

### Clusters hits

For each BLAST hit of HPI against the Clusters database we align both sequences and we measure the similarity between both sequences and record it as a Hit object in a dictionnary indexed by HPI gene names. We store the results as a pickle object.

In [10]:
result_handle = open(BLAST_HPI_CLUSTERS)
blast_records = NCBIXML.parse(result_handle)

muscle_cline = MuscleCommandline(input="algnt_temp.fasta", out="algnt_temp.txt")
results = dict()

for blast_record in blast_records:
    query = blast_record.query
    algnt = dict()
    refname = query.split(' ')[0]
    results[refname] = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps: 
            if hsp.expect < E_VALUE_THRESH:
                algnt[alignment.title.split(' ')[1]] = CLUSTERS_FASTA[alignment.title.split(' ')[1]]
    if(len(algnt.keys())>0):
        L = list(algnt.values())
        for i in range(len(L)):
            SeqIO.write([HPI_FASTA[refname]] + [L[i]],"algnt_temp.fasta","fasta") 
            muscle_cline()
            align = AlignIO.read("algnt_temp.txt", "fasta")
            refindex = 0
            hitindex = 1
            if(align[1,:].name==refname):
                refindex = 1
                hitindex = 0
            refseq = str(align[refindex,:].seq)
            hitseq = str(align[hitindex,:].seq)
            lref = len(refseq)
            global_sim = 0
            cluster_sim = 0
            start_gap = (hitseq[0]=="-")
            end_gap = (hitseq[-1]=="-")
            gaps = [hitseq[i]=="-" for i in range(len(hitseq))]
            start = gaps.index(False)
            end = len(hitseq)-gaps[::-1].index(False)
            for i in range(lref):
                if(refseq[i]==hitseq[i]):
                    global_sim += 1
                    if(i>=start and i<end):
                        cluster_sim += 1
            hit = Hit(refname, align[hitindex,:].name, global_sim/lref, cluster_sim/(end-start), start_gap, end_gap)
            results[refname].append(hit)

pickle.dump(results, open(RESULTS_CLUSTERS, 'wb'))     

We create a second dictionnary indexed by cluster names to store the results in a different order.

In [11]:
reverse_results = dict()
for key in results.keys():
    hits = results[key]
    for i in range(len(hits)):
        if(not hits[i].cluster_name in reverse_results.keys()):
            reverse_results[hits[i].cluster_name] = []
        reverse_results[hits[i].cluster_name].append(hits[i])

We output the results as a csv file.

In [14]:
file = open(ANNOTATIONS_CLUSTERS_CSV,"w")
file.write("Cluster,Cluster length,Gene name,Annotation,Gene length,Global similarity,Fragment,Local similarity,Start gap,End gap,Good alignment,Alignment length\n")

sorted_keys = []
for key in reverse_results.keys():
    sorted_keys += [int(key[7:])]
sorted_keys.sort()
for i in range(len(sorted_keys)):
    sorted_keys[i] = "cluster"+str(sorted_keys[i])
    
record = SeqIO.to_dict(SeqIO.parse(CLUSTERS_GAPS_FILE, "fasta"))
total = 0
total_good = 0
for key in sorted_keys:
    hits = reverse_results[key]
    sequence = [int(i=="-") for i in str(record[key].seq).strip("-")]
    a = sequence+[0]
    b = [0]+sequence
    c = [int((a[i]-b[i])==1) for i in range(len(a))]
    gaps = np.sum(np.array(c))
    algnt_length = len(str(record[key].seq))
    good_alignment = gaps<=max(MAX_GAPS_ABS,MAX_GAPS_PROP*np.sum(1-np.array(sequence)))
    total += 1
    total_good += int(good_alignment)
    for i in range(len(hits)):
        if(hits[i].is_similar() or hits[i].is_fragment()):
            line = hits[i].cluster_name[7:]+","
            line += str(hits[i].get_cluster_length())+","
            line += hits[i].get_gene_name()+","
            line += HPI_FASTA[hits[i].gene_name].description+","
            line += str(hits[i].get_gene_length())+","
            line += str(hits[i].global_similarity)+","
            line += str(hits[i].is_fragment())+","
            line += str(hits[i].cluster_similarity)+","
            line += str(hits[i].start_gap)+","
            line += str(hits[i].end_gap)+","
            line += str(good_alignment)+","
            line += str(algnt_length)+"\n"
            file.write(line)
file.close()

### Singletons annotations

For each BLAST hit of HPI against the Singletons database we align both sequences and we measure the similarity between both sequences and record it as a Hit object in a dictionnary indexed by HPI gene names. We store the results as a pickle object.

In [16]:
result_handle = open(BLAST_HPI_SINGLETONS)
blast_records = NCBIXML.parse(result_handle)

muscle_cline = MuscleCommandline(input="algnt_temp.fasta", out="algnt_temp.txt")
results = dict()

for blast_record in blast_records:
    query = blast_record.query
    algnt = dict()
    refname = query.split(' ')[0]
    results[refname] = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps: 
            if hsp.expect < E_VALUE_THRESH:
                algnt[alignment.title.split(' ')[1]] = SINGLETONS_FASTA[alignment.title.split(' ')[1]]
    if(len(algnt.keys())>0):
        L = list(algnt.values())
        for i in range(len(L)):
            SeqIO.write([HPI_FASTA[refname]] + [L[i]],"algnt_temp.fasta","fasta") 
            muscle_cline()
            align = AlignIO.read("algnt_temp.txt", "fasta")
            refindex = 0
            hitindex = 1
            if(align[1,:].name==refname):
                refindex = 1
                hitindex = 0
            refseq = str(align[refindex,:].seq)
            hitseq = str(align[hitindex,:].seq)
            lref = len(refseq)
            global_sim = 0
            cluster_sim = 0
            start_gap = (hitseq[0]=="-")
            end_gap = (hitseq[-1]=="-")
            gaps = [hitseq[i]=="-" for i in range(len(hitseq))]
            start = gaps.index(False)
            end = len(hitseq)-gaps[::-1].index(False)
            for i in range(lref):
                if(refseq[i]==hitseq[i]):
                    global_sim += 1
                    if(i>=start and i<end):
                        cluster_sim += 1
            hit = Hit(refname, align[hitindex,:].name, global_sim/lref, cluster_sim/(end-start), start_gap, end_gap)
            results[refname].append(hit)
pickle.dump(results, open(RESULTS_SINGLETONS, 'wb'))   

We create a second dictionnary indexed by cluster names to store the results in a different order.

In [17]:
reverse_results = dict()
for key in results.keys():
    hits = results[key]
    for i in range(len(hits)):
        if(not hits[i].cluster_name in reverse_results.keys()):
            reverse_results[hits[i].cluster_name] = []
        reverse_results[hits[i].cluster_name].append(hits[i])

We output the results as a csv file.

In [18]:
file = open(ANNOTATIONS_SINGLETONS_CSV,"w")
file.write("Cluster,Cluster length,Gene name,Annotation,Gene length,Global similarity,Fragment,Local similarity,Start gap,End gap\n")

sorted_keys = []
for key in reverse_results.keys():
    sorted_keys += [int(key[7:])]
sorted_keys.sort()
for i in range(len(sorted_keys)):
    sorted_keys[i] = "cluster"+str(sorted_keys[i])
    
for key in sorted_keys:
    hits = reverse_results[key]
    for i in range(len(hits)):
        if(hits[i].is_similar() or hits[i].is_fragment()):
            line = hits[i].cluster_name[7:]+","
            line += str(hits[i].get_cluster_length())+","
            line += hits[i].get_gene_name()+","
            line += K12_FASTA[hits[i].gene_name].description+","
            line += str(hits[i].get_gene_length())+","
            line += str(hits[i].global_similarity)+","
            line += str(hits[i].is_fragment())+","
            line += str(hits[i].cluster_similarity)+","
            line += str(hits[i].start_gap)+","
            line += str(hits[i].end_gap)+"\n"
            file.write(line)
file.close()

## Merging the results

Here we merge the results of HPI BLASTs against Clusters and Singletons databases.

In [21]:
results_clusters = pickle.load(open(RESULTS_CLUSTERS, 'rb'))
results_singletons = pickle.load(open(RESULTS_SINGLETONS, 'rb'))
results = dict()

for key in results_clusters.keys():
    results[key] = results_clusters[key]

for key in results_singletons.keys():
    if(key not in results.keys()):
        results[key] = results_singletons[key]
    else:
        results[key] += results_singletons[key]

file = open(ANNOTATIONS_HPI_CSV,"w")
file.write("Gene name,Annotation,Gene length,Cluster,Cluster length,Global similarity,Fragment,Local similarity,Start gap,End gap\n")

for key in results.keys():
    hits = results[key]
    for i in range(len(hits)):
        if(hits[i].is_similar() or hits[i].is_fragment()):
            line = hits[i].get_gene_name()+","
            line += HPI_FASTA[hits[i].gene_name].description+","
            line += str(hits[i].get_gene_length())+","
            line += hits[i].cluster_name[7:]+","
            line += str(hits[i].get_cluster_length())+","
            line += str(hits[i].global_similarity)+","
            line += str(hits[i].is_fragment())+","
            line += str(hits[i].cluster_similarity)+","
            line += str(hits[i].start_gap)+","
            line += str(hits[i].end_gap)+"\n"
            file.write(line)
file.close()

## Additionnal stuffs

In [7]:
results_clusters = pickle.load(open(RESULTS_CLUSTERS, 'rb'))
results_singletons = pickle.load(open(RESULTS_SINGLETONS, 'rb'))

In [23]:
reverse_results_singletons = dict()
for key in results_singletons.keys():
    hits = results_singletons[key]
    for i in range(len(hits)):
        if(hits[i].is_similar() or hits[i].is_fragment()):
            if(not hits[i].cluster_name in reverse_results_singletons.keys()):
                reverse_results_singletons[hits[i].cluster_name] = []
            reverse_results_singletons[hits[i].cluster_name].append(hits[i])

reverse_results_clusters = dict()
for key in results_clusters.keys():
    hits = results_clusters[key]
    for i in range(len(hits)):
        if(hits[i].is_similar() or hits[i].is_fragment()):
            if(not hits[i].cluster_name in reverse_results_clusters.keys()):
                reverse_results_clusters[hits[i].cluster_name] = []
            reverse_results_clusters[hits[i].cluster_name].append(hits[i])

In [25]:
gene_id = []
cluster_id = []
gene_length = []
fragment = []
singleton = []
bad_algnt = []
gaps = []
dashes = []

for key in reverse_results_singletons.keys():
    hits = reverse_results_singletons[key]
    if(len(hits)==1):
        gene_id.append(hits[0].gene_name.split("gene_")[1])
        cluster_id.append(hits[0].cluster_name[7:])
        gene_length.append(hits[0].get_gene_length())
        fragment.append(hits[0].is_fragment())
        singleton.append("True")
        bad_algnt.append("")
        gaps.append("")
        dashes.append("")

for key in reverse_results_clusters.keys():
    hits = reverse_results_clusters[key]
    if(len(hits)==1):
        gene_id.append(hits[0].gene_name)
        cluster_id.append(hits[0].cluster_name[7:])
        gene_length.append(str(hits[0].get_gene_length()))
        fragment.append(str(hits[0].is_fragment()))
        singleton.append("False")
        sequence = [int(i=="-") for i in str(CLUSTERS_FASTA[hits[0].cluster_name].seq)]
        a = sequence+[0]
        b = [0]+sequence
        c = [int((a[i]-b[i])==1) for i in range(len(a))]
        gaps_count = np.sum(np.array(c))
        characters = len(sequence)-np.sum(np.array(sequence))
        if(gaps_count>max(MAX_GAPS_PROP*characters,MAX_GAPS_ABS)):
            bad_algnt.append("True")
            gaps.append(str(gaps_count))
            dashes.append(str(np.sum(np.array(sequence))))
        else:
            bad_algnt.append("False")
            gaps.append("")
            dashes.append("")
        
df = pd.DataFrame({
    "gene_id":gene_id,
    "cluster_id":cluster_id,
    "gene_length":gene_length,
    "fragment":fragment,
    "singleton":singleton,
    "bad_algnt":bad_algnt,
    "gaps":gaps,
    "dashes":dashes
})
     
df.to_csv(UNIQUE_HPI_HITS_CSV,index=False)